# Channel and Marker clear test


# Description:
This script test the functionality to clear channel memory,clearing the channel will clear out the marker's output. Wer will observe the channel's and marker's output become zero as we clear the channel memory.

# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 1 SMA to BNC Cable
4. 1 SMP to BNC Cable

# Hardware Connection
    Proteus Module    --->     Oscilloscope
    As per the script

# Test Procedure
1. Connect a channel to oscilloscope channel 1 and each marker separately and perform the test for all marker with respective channels individually.
2. Run the script and observe output on scope.

Proteus release - Aurora

In [3]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('..//..//SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

In [ ]:
# Connect Proteus 
ip_address = '192.90.70.22'
inst = connect(ip_address)
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)
inst.default_paranoia_level = 2
# Reset the instrument
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':TRACe:DELete:ALL')
print('Restarting '+resp)
# Get the model name
model = inst.send_scpi_query(":SYST:iNF:MODel?")
print("Model: " + model)
# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    channels_per_dac = 2
else:
    dac_mode = 8
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
    channels_per_dac = 1
half_dac = max_dac / 2.0
print('DAC {0} bits'.format(dac_mode))
resp = inst.send_scpi_query(':SYST:ERR?')
if not resp.startswith('0'):
    print("ERROR1", resp)
# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)
# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
max_seg_number = int(resp)
print("Max segment number: {}".format(max_seg_number))

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))
########################################################################################################################
# Build wave-data and markers-data
print('Build wave-data and markers-data ..')
test_success = True
seg_wave_points = 4096
ncycles = 1
cyclelen = seg_wave_points / ncycles

wave = [None]
mark = [None]
# Buffer size for the waveform
if dac_mode == 16:
    seg_wave_bytes = seg_wave_points * 2  # each point is 2 bytes in size
else:
    seg_wave_bytes = seg_wave_points
# seg_wave_bytes = seg_wave_points * 2 # each waveform point is represented as uint16 (2 bytes in size)
sampling_rate = 2500e6
# Buffer size for Marker
if dac_mode == 16 and sampling_rate <= 2500000000:
    marker_cycle = 2
    seg_mark_bytes = seg_wave_points // 4
else:
    marker_cycle = 8
    seg_mark_bytes = seg_wave_points // 8
x = np.linspace(start=0, stop=seg_wave_points, num=seg_wave_points, endpoint=False)
yw = np.fmod(x, cyclelen)
yw = (yw <= cyclelen / 2) * max_dac
yw = np.round(yw)
yw = np.clip(yw, 0, max_dac)
yw = yw.astype(wpt_type)
yw.reshape(-1)  # = yw.astype(data_type)
# Build marker
x = np.linspace(
    start=0, stop=seg_mark_bytes, num=seg_mark_bytes, endpoint=False)
y = np.fmod(x, seg_mark_bytes)
y = (y <= seg_mark_bytes / 2) * 0x33
y = np.round(y)
y = np.clip(y, 0, 255)
# y= np.ones(seg_mark_bytes,np.uint8) *0xFF
mark = y.astype(np.uint8).reshape(-1)
del x, y
segnum = 1
wav = yw
mrk = mark

########################################################################################################################
scope_addr = 'USB0::0x2A8D::0x900E::MY55490134::INSTR' # connect to scope via USB
try:
    resourceManager = visa.ResourceManager()  # Create a connection (session) to the instrument
    #scope = resourceManager.get_instrument(scope_addr2)
    #scope.write('*CLS;:DISPlay:CGRade:LEVels ')
    scope = resourceManager.open_resource(scope_addr)
    print(scope)
    ## scope acquisition
    # Send *IDN? and read the response
    scope.write('*RST')
    scope.write('*IDN?')
    idn = scope.read()
    print('*IDN? returned: %s' % idn.rstrip('\n'))
except Error as ex2:
    print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr)
    sys.exit()

########################################################################################################################
vertical_scale_per_divison = 0.062
horizontal_scale_per_divison = 100e-12
rise_time = 200e-12
mrkr_on_scope=2
for ii in range(4):
    channb = ii+1
    for i in range(2):
        mrkr =  i+1
        print('Connect Channel {0} and its Marker {1} to channel 1 and 2 of the oscilloscope'
          '\nPress enter '.format(channb, mrkr))
        input()
        print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
        print('Number of wave point:{0}'.format(seg_wave_points))
        inst.send_scpi_cmd(':INST:CHAN {0}'.format(channb))  # Select channel
        inst.send_scpi_cmd(':TRAC:DEF {0},{1}'.format(segnum, seg_wave_points))  # Define segment
        inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))  # Select the segment
        inst.timeout = 30000  # Increase the timeout before writing binary-data:
        # Send the binary-data with *OPC? added to the beginning of its prefix.
        # inst.write_binary_values('*OPC?; :TRAC:DATA', wav, datatype=wav_dat_type)
        inst.write_binary_data(':TRAC:DATA', wav)
        # Read the response to the *OPC? query that was added to the prefix of the binary data
        # resp = inst.read()
        # Set normal timeout
        inst.timeout = 10000
        inst.send_scpi_cmd(':INST:CHAN {0}'.format(channb))  # Select channel
        inst.send_scpi_cmd('FUNC:MODE ARB')
        inst.send_scpi_cmd(
            ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum))  # Play the specified segment at the selected channel
        inst.send_scpi_cmd(':OUTP OFF')
        inst.send_scpi_cmd(':SOUR:VOLT 0.5')
        # Turn on the output of the selected channel:
        inst.send_scpi_cmd(':OUTP ON')
        print('Download markers to segment {0} of channel {1}'.format(segnum, channb))
        inst.timeout = 30000# Increase the timeout before writing binary-data:
        # Send the binary-data with *OPC? added to the beginning of its prefix.
        inst.send_scpi_cmd(':INST:CHAN {0}'.format(channb))
        inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))
        inst.write_binary_data(':MARK:DATA', mark)
        # Read the response to the *OPC? query that was added to the prefix of the binary data
        # resp = inst.read()
        inst.timeout = 10000 # Set normal timeout
        inst.send_scpi_cmd(':INST:CHAN {0}'.format(channb))
        inst.send_scpi_cmd(':MARK:SEL {};:MARK:STAT ON'.format(mrkr))
        channb = inst.send_scpi_query(':INST:CHAN?')
        mrker = inst.send_scpi_query(':MARK:SEL?')
        inst.send_scpi_cmd(':MARK:SEL 2;:MARK:STAT ON')
        inst.send_scpi_cmd(':MARK:VOLT:PTOP 0.5;:MARK:VOLT:OFFS 0')
        scope.write('AUTOscale')
        time.sleep(5)
        scope.write('*OPC')
        scope.write('*CLS;:DISPlay:CGRade:LEVels')
        scope.write(':MEASure:CLEar')
        scope.write(':MEASure:VPP CHANnel1')
        time.sleep(1)
        scope.write(':MEASure:RESults?')
        result = scope.read()
        vpp_ch1i = float(result.split(',')[2])
        scope.write(':MEASure:VPP CHANnel2')
        time.sleep(1)
        scope.write(':MEASure:RESults?')
        result = scope.read()
        vpp_ch1m1i = float(result.split(',')[2])
        if vpp_ch1i >= 0.5 and vpp_ch1m1i >= 0.5:
            inst.send_scpi_cmd(':INST:CHAN {0};:TRAC:ZERO:ALL'.format(channb))
            time.sleep(5)
            scope.write(':MEASure:CLEar')
            scope.write(':MEASure:VPP CHANnel1')
            time.sleep(1)
            scope.write(':MEASure:RESults?')
            result = scope.read()
            vpp_ch1 = float(result.split(',')[2])

            scope.write(':MEASure:VPP CHANnel2')
            time.sleep(1)
            scope.write(':MEASure:RESults?')
            result = scope.read()
            vpp_ch1m1 = float(result.split(',')[2])
            if (vpp_ch1)<0.1 and (vpp_ch1m1)<0.1:
                print('Test pass for channel {0} marker {1}'.format(channb, mrkr))
            else:
                print('Test FAIL for channel {0} marker {1}'.format(channb, mrkr))
        else:
            print("Please check connections")
if (test_success):
    print('Test successed for marker clear test ')
else:
    print('Test failed for marker clear test ')
disconnect()

Service connect
Trying to connect to IP:192.90.70.22
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000002232810,1.238.6 --slot#: 2, slot#: 4, 
Restarting Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000002232810,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M
DAC 16 bits
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes
Build wave-data and markers-data ..
USBInstrument at USB0::0x2A8D::0x900E::MY55490134::0::INSTR
*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101
Connect Channel 1 and its Marker 1 to channel 1 and 2 of the oscilloscope
Press enter 

Download wave to segment 1 of channel 1
Number of wave point:4096
Download markers to segment 1 of channel 1
Test pass for channel 1 marker 1
Connect Channel 1 and its Marker 2 to channel 1 and 2 of the oscilloscope
Press enter 
